# MQTT Subscribe-Publish Testing Environment

## INTRODUCTION

This notebook demonstrates the functionality of `publish` and `subscribe` using:
- Several different broker types:
    - Local Mosquitto MQTT broker (without credentials)
    - Remote AIMPF MQTT broker (with credentials)
- The `publish` decorator to send messages in both **synchronous** and **asynchronous** modes
    - `@publish("test")`
    - `@async_publish("test")`
- The `subscribe` decorator to process messages in both **synchronous** and **asynchronous** modes
    - `@subscribe("test")`
    - `@async_subscribe("test")`

### Prerequisites
- **General**
    - Ensure `pycarta` is installed in editable mode (`pip install -e .`) from the root directory of the project.

- **Local Mosquitto MQTT broker**
    - Ensure Mosquitto is installed and running:
        ```bash
        brew install mosquitto
        mosquitto
        ```
    - A mosquitto subscriber can be set up as follows to listen to topic `test`:
        ```bash
        mosquitto_sub -t test
        ```
    - A mosquitto publisher can publish `$YOUR_MESSAGE` to topic `test` as follows:
        ```bash
        mosquitto_pub -t test -m $YOUR_MESSAGE
        ```

- **Remote AIMPF MQTT broker**
    - A set of credentials available including:
        - broker_address (not in the certificate requirement, but will be needed for connection)
        - ca_certificate
        - private_key
        - certificate
    - Basic information about the broker, e.g., broker address, port, etc.
    - (Optional, but highly recommended) A Carta account for credential CRUD operations 
        - You have already uploaded MQTT credentials to your chosen `tag`. Please check the Jupyter Notebook named `credential-crud.ipynb` for more details.
        - Environment variables set for authentication for your account (e.g., `CARTA_CHEN_USERNAME`, `CARTA_CHEN_PASSWORD`).


In [2]:
import time
import os
import logging
import random
from dotenv import load_dotenv

load_dotenv(override=True)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logging.basicConfig()

def ri(lower=1, upper=100):
    return random.randint(lower, upper)

from dotenv import load_dotenv
from pycarta.mqtt.subscriber import subscribe
from pycarta.mqtt.publisher import publish
import logging

# Load environment variables
load_dotenv(override=True)

# Synchronous publisher using default configuration
@publish(
    topic="sensor/default",
    host="test.mosquitto.org",
    port=1883
)
def p_default_publisher():
    return {"message": "Hello from publisher using default constants"}

# Synchronous subscriber with a 6-second polling timeout
@subscribe(
    topic="sensor/data_6s",
    host="test.mosquitto.org",
    port=1883,
    config={"POLLING_TIMEOUT": 6}
)
def s1_subscriber(msg):
    return msg

# Synchronous publisher with a 5-second connection timeout
@publish(
    topic="sensor/data_5s",
    host="test.mosquitto.org",
    port=1883,
    config={"CONNECTION_TIMEOUT": 5}
)
def p1_publisher():
    return {"message": "Hello from publisher 1"}

# Publisher using configuration loaded from an INI file.
# (Assume the INI file is located at "../../tests/mqtt/custom_config.ini")
@publish(
    topic="sensor/ini_config",
    host="test.mosquitto.org",
    port=1883,
    config="../../tests/mqtt/custom_config.ini"
)
def p_ini_config_publisher():
    return {"message": "Hello from publisher using INI configuration"}

# ==== Demonstrate configuration retrieval ====
# Because the decorated functions are now callable wrapper class instances,
# they have a 'config' attribute.

print("\n=== Checking Configuration ===")

print("\nPublisher using default constants:")
print(p_default_publisher.config)

print("\nSubscriber 1 (15s polling timeout):")
print(s1_subscriber.config)

print("\nPublisher 1 (5s connection timeout):")
print(p1_publisher.config)

print("\nPublisher using INI configuration:")
print(p_ini_config_publisher.config)

p_default_publisher()
# s1_subscriber()

In [3]:
# NOTE: Here, we are trying to make the brokers definition consistent between Jupyter Notebook and Pytest.
BROKERS = [
    {
        "label": "mosquitto_local",
        "host": "localhost",
        "port": 1883,
    },
    {
        "label": "aimpf_remote_cred",
        "host": "aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com",
        "port": 8883,
    },
]

# Create a dictionary for quick access by label
brokers_dict = {broker["label"]: broker for broker in BROKERS}

# NOTE: Before you proceed, select the testing broker by its label
# testing_broker = brokers_dict["mosquitto_local"]
testing_broker = brokers_dict["aimpf_remote_cred"]



In [4]:
import time
import os
import logging
import random
from pycarta import login
from pycarta.mqtt.publisher import publish
from pycarta.mqtt.subscriber import subscribe
from pycarta.mqtt.credential import BilateralCredentialAuthenticator

logger = logging.getLogger(__name__)

if testing_broker["label"] == "aimpf_remote_cred":
    # Environment variables
    CARTA_USERNAME = os.getenv("CARTA_CHEN_USERNAME")
    CARTA_PASSWORD = os.getenv("CARTA_CHEN_PASSWORD")
    CARTA_HOST = os.getenv("CARTA_HOST_DEV", "https://api.sandbox.carta.contextualize.us.com")
    TAG = "mb33-thing301"
    certificate_folder = "../../tests/mqtt/mb33-thing301"
    ca_cert_file = "../../tests/mqtt/mb33-thing301/AmazonRootCA1.pem"
    client_cert_file = "../../tests/mqtt/mb33-thing301/34c68a3ca20dfafa5a327c03529649d206b13a789c08df92be63bbea2514cb3d-certificate.pem.crt"
    client_key_file = "../../tests/mqtt/mb33-thing301/34c68a3ca20dfafa5a327c03529649d206b13a789c08df92be63bbea2514cb3d-private.pem.key"

    # Approach #1: Provide 3 file paths
    # auth = BilateralCredentialAuthenticator.from_cert_files(
    #     ca_cert=ca_cert_file,
    #     cert=client_cert_file,
    #     key=client_key_file
    # )
    # logger.info("Approach #1 authenticator created.")

    # Approach #2: Provide 1 folder path containing exactly 3 PEM files
    # auth = BilateralCredentialAuthenticator.from_folder(certificate_folder)
    # logger.info("Approach #2 authenticator created.")

    # Approach #3: Provide username/password/tag to retrieve from Carta
    login(username=CARTA_USERNAME, password=CARTA_PASSWORD, host=CARTA_HOST)
    auth = BilateralCredentialAuthenticator.from_carta(tag=TAG)
    logger.info("Approach #3 authenticator created via Carta tag '%s'.", TAG)

    logger.info("BilateralCredentialAuthenticator initialized!")
    


INFO:__main__:Approach #3 authenticator created via Carta tag 'mb33-thing301'.
INFO:__main__:BilateralCredentialAuthenticator initialized!


---
## SECTION 1: Synchronous Mode

### Example 1.1: Publish Results of a Function
This example demonstrates publishing results to a topic using the `publish` decorator.

# Use the publish decorator on a simple function
if testing_broker["label"] == "mosquitto_local":
    @publish(
        "test",
        host=testing_broker["host"])
    def add(lhs, rhs):
        return lhs + rhs
elif testing_broker["label"] == "aimpf_remote_cred":
    @publish(
        "test",
        host=testing_broker["host"],
        port=testing_broker["port"],
        qos=1,
        authenticator=auth)
    def add(lhs, rhs):
        return lhs + rhs

logger.info(f"Function `add` has been defined for testing against broker `{testing_broker["label"]}`")

# Publish results
add(12, 30)  # Publishes 42

# Clean up
del add

### Example 1.2: Subscribe to a Topic and Process Messages
This example demonstrates subscribing to the topic and processing messages using the `subscribe` decorator.

In [5]:

if testing_broker["label"] == "mosquitto_local":
    @subscribe("test")
    def double(x):
        return x * 2
elif testing_broker["label"] == "aimpf_remote_cred":
    @subscribe(
        topic="test",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    def double(x):
        return float(x) * 2

logger.info(f"Function `double` has been defined for testing against broker `{testing_broker["label"]}`")

# Single message processing (simulate publisher: mosquitto_pub -t test -m 25)
# The subscriber will stop if:
# 1. Receive a published message
# 2. Timeout (TimeoutException after certain amount of time)
# 3. Canceled (KeyboardInterrupt)
double()

INFO:__main__:Function `double` has been defined for testing against broker `aimpf_remote_cred`
DEBUG:pycarta.mqtt.connection:[Sync Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
INFO:pycarta.mqtt.connection:[Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883...
INFO:pycarta.mqtt.connection:[Connection] Connected to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.subscriber:[Sync subscriber] Subscribing to topic test
DEBUG:pycarta.mqtt.connection:[Sync Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


### Example 1.3: Subscribe and Process Multiple Messages
This example shows how to process multiple messages until a `KeyboardInterrupt` or polling timeout occurs.

# Process messages until KeyboardInterrupt or timeout (default: 30 seconds)
[x for x in double]

#

---
## SECTION 2: Asynchronous Mode

import logging
import os
from dotenv import load_dotenv
logging.basicConfig(
    level=logging.DEBUG, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
load_dotenv(override=True)
# print(f"POLLING_TIMEOUT: {os.getenv('POLLING_TIMEOUT')}")
# print(f"POLLING_INTERVAL: {os.getenv('POLLING_INTERVAL')}")

In [6]:
import asyncio
from pycarta.mqtt.publisher import publish
from pycarta.mqtt.subscriber import subscribe

# Define the asynchronous publisher and subscriber functions
if testing_broker["label"] == "mosquitto_local":
    @publish("test",
        host=testing_broker["host"],
        port=testing_broker["port"])
    async def add(lhs, rhs):
        return lhs + rhs
elif testing_broker["label"] == "aimpf_remote_cred":
    @publish(
        topic="test",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    async def add(lhs, rhs):
        return lhs + rhs

logger.info(f"Async function `add` has been defined for testing against broker `{testing_broker["label"]}`")

# Define the asynchronous subscriber function to yield
if testing_broker["label"] == "mosquitto_local":
    @subscribe("test",
        host=testing_broker["host"],
        port=testing_broker["port"])
    async def double(x):
        return x * 2
elif testing_broker["label"] == "aimpf_remote_cred":
    @subscribe(
        topic="test",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    async def double(x):
        print("Received message:", x)
        return x * 2

logger.info(f"Async function `double` has been defined for testing against broker `{testing_broker["label"]}`")

INFO:__main__:Async function `add` has been defined for testing against broker `aimpf_remote_cred`
INFO:__main__:Async function `double` has been defined for testing against broker `aimpf_remote_cred`


In [7]:
await add(ri(), ri())

DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'test'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


78

@publish(topic="test", host="localhost", port=1883)
async def async_pub():
    await asyncio.sleep(1)
    return f"Hello from async_pub: {random.randint(1, 100):03d}."

await async_pub()
# asyncio.run(async_pub())

In [8]:
await double()

DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.subscriber:[Async subscriber] Subscribing to topic test
DEBUG:pycarta.mqtt.subscriber:[Async subscriber] No message received within timeout period
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


In [9]:
subscriber_iter = double.__aiter__()

In [10]:
await double()

DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.subscriber:[Async subscriber] Subscribing to topic test
DEBUG:pycarta.mqtt.subscriber:[Async subscriber] No message received within timeout period
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


In [11]:
double.cancel()

In [12]:
# This will run the asynchronous publisher in non-blocking mode in JN
[m async for m in double]

DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.subscriber:[Async subscriber] Subscribing to topic test for iteration
DEBUG:pycarta.mqtt.subscriber:[Async subscriber] No message received within timeout period
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[]

### Example 2.1: Asynchronous Calculation Publish and Subscribe

#### Overview
This script demonstrates an asynchronous MQTT-based publish and subscribe system for simple calculations, leveraging Python's `asyncio` for efficient, non-blocking execution.

#### Process
- **Publish**:
  - Asynchronously calculates the sum of two numbers.
  - Publishes the result to the MQTT topic `test`.
  - Sends multiple messages sequentially with a delay between each.

- **Subscribe**:
  - Asynchronously listens to the `test` topic for incoming messages.
  - Processes each message by doubling its value and printing the result.
  - Stops automatically when a processed value exceeds 20, demonstrating event-driven control.


In [12]:
import asyncio
from pycarta.mqtt.publisher import publish
from pycarta.mqtt.subscriber import subscribe


if testing_broker["label"] == "mosquitto_local":
    # Define the asynchronous publisher and subscriber functions
    @publish("test")
    async def add(lhs, rhs):
        return lhs + rhs

    # Define the asynchronous subscriber function to yield
    @subscribe("test")
    async def double(x):
        return x * 2
elif testing_broker["label"] == "aimpf_remote_cred":
    # Define the asynchronous publisher and subscriber functions
    @publish(
        topic="test",
        host=testing_broker["host"],
        port=testing_broker["port"],
        qos=1,
        authenticator=auth)
    async def add(lhs, rhs):
        return lhs + rhs

    # Define the asynchronous subscriber function to yield
    @subscribe(
        topic="test",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    async def double(x):
        print(f"[Subscriber] Received message: {x}")
        result = int(x) * 2
        print(f"[Subscriber] Doubled value: {result}")
        return result

async def main():
    print("[Main] Starting MQTT subscription to 'test' topic...")
    
    # Initialize a list to store received messages
    received_messages = []
    
    # Start the publisher as a background task
    publisher_task = asyncio.create_task(publish_messages())
    print("[Main] Publisher task started in the background. Subscriber will process the delayed incoming messages.")
    
    # Process incoming messages from the subscriber using async for
    async for message in double:
        if message is not None:
            print(f"[Main] Processed message result: {message}")
            # Append each received message to the list
            received_messages.append(message)
            # Exit condition for demonstration purposes
            if message > 20:
                print("[Main] Received a message greater than 20. Exiting.")
                break
    
    # Await the publisher task to ensure all messages are published
    await publisher_task
    
    print("[Main] Subscription ended.")
    
    # Access the list of received messages after exiting the loop
    print(f"[Main] All received messages: {received_messages}")

async def publish_messages():
    # List of values to publish
    test_values = [0, 5, 10, 15, 20]
    
    # Publish messages sequentially
    for value in test_values:
        await asyncio.sleep(1)   # Short delay between publishes
        await add(value, value)  # For example, publish lhs + rhs
        await asyncio.sleep(1)   # Short delay between publishes
    
    print("[Publisher] Published all messages.")

# Execute the main coroutine and the expected final results are: 
# >>> [Main] All received messages: [0, 20, 40]
await main()

DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Main] Starting MQTT subscription to 'test' topic...
[Main] Publisher task started in the background. Subscriber will process the delayed incoming messages.


DEBUG:pycarta.mqtt.subscriber:[Async subscriber] Subscribing to topic test for iteration
DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'test'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Subscriber] Received message: 0
[Subscriber] Doubled value: 0
[Main] Processed message result: 0


DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'test'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Subscriber] Received message: 10
[Subscriber] Doubled value: 20
[Main] Processed message result: 20


DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'test'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Subscriber] Received message: 20
[Subscriber] Doubled value: 40
[Main] Processed message result: 40
[Main] Received a message greater than 20. Exiting.


DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'test'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'test'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Publisher] Published all messages.
[Main] Subscription ended.
[Main] All received messages: [0, 20, 40]


In [13]:
# NOTE: If you run the previous cell, any message published after the subscriber is stopped will remain unprocessed in the queue. As a result, when the subscriber is resumed, the stale messages will still be processed. For example, if the subscriber processes numbers and the previously processed results are [60, 80, 2, 4, 6], newly published numbers 1, 2, 3 in separate publishing events would unexpectedly append their processed results to the old results, leading to [60, 80, 2, 4, 6] instead of the expected [2, 4, 6].
logger.info(f"Queue before clearing: {list(double.result_queue._queue)}")

# To address this issue, you must clear the queue explicitly using one of the following approaches before resuming or stopping the subscriber:

# 1. Clear the queue explicitly:
await double.clear_queue()

# 2. Disconnect the subscriber and clear the queue simultaneously:
# await double.disconnect_async(clear_queue=True)

# Simply disconnecting the subscriber without clearing the queue will not remove the residual messages, and they will persist:
# await double.disconnect_async()

# To confirm the contents of the queue at any point, you can inspect it directly:
logger.info(f"Queue after clearing: {list(double.result_queue._queue)}")

AttributeError: 'SubscribeWrapper' object has no attribute 'result_queue'

In [ ]:
# After running the cleanup method, the result queue should be empty before new subscriber is launched, and the result should be [2, 4, 6] as expected now.
# await double()
# [message async for message in double]

### Example 2.2: Asynchronous Sensor Monitoring and Control

#### Overview
This script demonstrates an asynchronous MQTT-based system for monitoring sensor values and triggering control actions based on predefined thresholds.

#### Process
- **Publish**:
  - Sends simulated sensor values to the MQTT topic **`machine/sensor`**.
  - Calculates and publishes the sum of two numbers as the sensor value.
  - Provides error (`STOP`) or no-error (`CONTINUE`) status messages based on thresholds.

- **Subscribe**:
  - Listens to the **`machine/sensor`** topic for sensor values.
  - Processes incoming messages and evaluates them against defined thresholds.
  - Publishes a **`STOP`** action if the value is out of bounds, halting further processing.
  - Publishes a **`CONTINUE`** action if the value is within bounds, allowing normal operation.

- **Main**:
  - Manages asynchronous publishing and subscribing with `asyncio`.
  - Ensures sensor values are monitored in real-time and appropriate control actions are taken.
  - Ends the subscription after an out-of-bound value triggers a **`STOP`** action.
  

In [10]:
import asyncio
import json
from pycarta.mqtt.publisher import publish
from pycarta.mqtt.subscriber import subscribe

# Define expected value and variance
mu = 12.34
sigma = 3.45
lower_bound, upper_bound = (mu - 2.0 * sigma, mu + 2.0 * sigma)

if testing_broker["label"] == "mosquitto_local":
    # Define the asynchronous publisher functions
    @publish(topic="machine/sensor")
    async def add(lhs, rhs):
        return lhs + rhs

    @publish(topic="machine/sensor")
    async def on_error():
        return "STOP"

    @publish(topic="machine/sensor")
    async def no_error():
        return "CONTINUE"

    # Define the asynchronous subscriber function to yield sensor value
    @subscribe("machine/sensor")
    async def get_sensor_value(x):
        return x
elif testing_broker["label"] == "aimpf_remote_cred":
    # Define the asynchronous publisher functions
    @publish(
        topic="machine/sensor",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    async def add(lhs, rhs):
        return lhs + rhs

    @publish(
        topic="machine/sensor",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    async def on_error():
        return "STOP"

    @publish(
        topic="machine/sensor",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    async def no_error():
        return "CONTINUE"

    # Define the asynchronous subscriber function to yield sensor value
    @subscribe(
        topic="machine/sensor",
        host=testing_broker["host"],
        port=testing_broker["port"],
        authenticator=auth)
    async def get_sensor_value(x):
        try:
            x = float(x)
        except ValueError:
            print(f"[Subscriber] Received non-numeric sensor value: {x}. Ignoring.")
            return None
        return x


# Define the main coroutine
async def main():
    print("[Main] Starting MQTT subscription to 'machine/sensor' topic...")
    
    # Initialize a list to store received messages
    received_messages = []
    
    # Start the publisher as a background task
    publisher_task = asyncio.create_task(publish_messages())
    print("[Main] Publisher task started in the background. Subscriber will process the incoming messages.")
    
    # Process incoming messages from the subscriber using async for
    async for x in get_sensor_value:
        if x is not None:
            print(f"[Main] Received sensor value: {x}")
            
            # Check if x can be converted to a number
            try:
                x_float = float(x)
                received_messages.append(x)  # Append each received message to the list
            except ValueError:
                print(f"[Main] Received non-numeric sensor value: {x}. Ignoring.")
                continue

            # Check error condition
            if not lower_bound < x < upper_bound:
                await on_error()
                print(f"[Main] Published 'STOP' action: value out of bounds [{lower_bound:.2f}, {upper_bound:.2f}]. ")
                break
            else:
                await no_error()
                print("[Main] Published 'CONTINUE' action.")
    
    # Await the publisher task to ensure all messages are published
    await publisher_task
    
    print("[Main] Subscription ended.")
    
    # Access the list of received messages after exiting the loop
    print(f"[Main] All received messages: {received_messages}")

# Define the publisher coroutine
async def publish_messages():
    # List of values to publish
    test_values = [7, 9, 11, 13]
    
    # Publish messages sequentially with a 3-second delay between publishes
    for value in test_values:
        await asyncio.sleep(3)   # Short delay to allow the subscriber to establish connection
        result = value + value  # Example: publish lhs + rhs (e.g., 5 + 5 = 10)
        await add(value, value)  # Publish the result to 'machine/sensor'
        print(f"[Publisher] Published sensor value: {result}")
        await asyncio.sleep(2)   # Short delay between publishes
    
    print("[Publisher] Published all messages.")

# Execute the main coroutine and the expected final results are: 
# >>> [Main] All received messages: [14, 18, 22]
# NOTE: After receiving the last number 22 which is actually out of range, the subscriber stops recording messages.
await main()


DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Main] Starting MQTT subscription to 'machine/sensor' topic...
[Main] Publisher task started in the background. Subscriber will process the incoming messages.


DEBUG:pycarta.mqtt.subscriber:[Async subscriber] Subscribing to topic machine/sensor for iteration
DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'machine/sensor'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Publisher] Published sensor value: 14


DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


...

[Main] Received sensor value: 22.0
[Main] Published 'STOP' action: value out of bounds [5.44, 19.24]. 


DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'machine/sensor'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883
DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Publisher] Published sensor value: 26


DEBUG:pycarta.mqtt.publisher:[Async Publisher] Publishing to topic 'machine/sensor'
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from aop63bhe1nwsr-ats.iot.us-east-1.amazonaws.com:8883


[Publisher] Published all messages.
[Main] Subscription ended.
[Main] All received messages: [14.0, 18.0, 22.0]
